In [1]:
import os
os.getpid()

24072

In [2]:
import json
f = open('ESConv.json')
data = json.load(f)
f.close()
processed_data = []
for i in data:
    pt = i['problem_type']
    for j in i['dialog']:
        processed_data.append([pt, j['content'], j['speaker'], j['annotation']])
sl = []
for i in processed_data:
    sl.append(len(i[1].split()))
## Safely, we remove length 1,2,3
to_remove = []
for i in range(len(sl)):
    if sl[i] <= 3:
        to_remove.append(i)
for index in sorted(to_remove, reverse=True):
    del processed_data[index]
# MERGE Issue with parents
for i in range(len(processed_data)):
    if processed_data[i][0] == 'conflict with parents':
        processed_data[i][0] = 'Issues with Parents'
# MERGE Into Family Issue
for i in range(len(processed_data)):
    if processed_data[i][0] == 'Issues with Parents':
        processed_data[i][0] = 'Family Issues'
    if processed_data[i][0] == 'Issues with Children':
        processed_data[i][0] = 'Family Issues'
# For now, we care the all 7 categories.
kept = ['problems with friends', 'ongoing depression','breakup with partner', 'academic pressure', 'job crisis', 'Family Issues','Sleep Problems']

filtered_processed_data = []
for i in processed_data:
    if i[0] in kept:
        filtered_processed_data.append(list(i))
        filtered_processed_data[-1][0] = kept.index(filtered_processed_data[-1][0])
import random

random.seed(10)
random.shuffle(filtered_processed_data)
lenth = len(filtered_processed_data)
train_list = filtered_processed_data[0:int(lenth*0.8)]
test_list = filtered_processed_data[int(lenth*0.8):]

from collections import Counter
# Count labels in training
lt = []
for i in test_list:
    lt.append(i[0])
Counter(lt)

Counter({2: 1333, 1: 1879, 3: 865, 4: 1489, 0: 935, 5: 157, 6: 137})

In [3]:
import torch
from torch.utils.data.sampler import BatchSampler, Sampler, RandomSampler, SequentialSampler
from torch.utils.data import Dataset, DataLoader

from transformers import BertTokenizer
from transformers import BertForSequenceClassification, BertModel, AdamW, BertConfig
from tqdm import tqdm

class Corpora():
    """
        A class that holds the texts and class labels.
    """
    
    # Modification: l represent train_list or test_list
    
    def __init__(self, l):
        """
            file_path: on MAGIC, it is either
                        data-badassnlp/cola_public/raw/in_domain_train.tsv or
                        data-badassnlp/cola_public/raw/in_domain_dev.tsv
                    on Google Colab, change them to the corresponding paths.
        """

        self.sentences = [s[1] for s in tqdm(l)]
        self.labels = [int(s[0]) for s in tqdm(l)]
        
        
class TextClassificationDataSet(Dataset):
    """
        Define a dataset consisting of pairs of (sequence_of_word_indices, class_label).
        class_label is either 0 or 1.
    """
    def __init__(self, corpora, tokenizer: BertTokenizer):

        self.corpora = corpora
        self.tokenizer = tokenizer

        d =  tokenizer.batch_encode_plus(corpora.sentences)
        
        padded_encoded_inputs = tokenizer.pad(d)
        
        self.input_ids = torch.tensor(padded_encoded_inputs.input_ids)
        self.attention_mask = torch.tensor(padded_encoded_inputs.attention_mask)
        self.labels = torch.tensor(corpora.labels)
        
    def __len__(self):
        return len(self.corpora.sentences)

    def __getitem__(self, idx):
        """
            Return the idx-th of the rows the self.input_ids, self.attention_mask, self.labels in this order.
            Don't do BERT tokenization here as that will be slow.
        """
        return self.input_ids[idx], self.attention_mask[idx], self.labels[idx]

In [4]:
from torch import nn

class BERTClassifier(nn.Module):
    
    def __init__(self, BERT_model, hidden_layer_size, num_classes):
        
        super(BERTClassifier, self).__init__()

        # loaded pretrained model
        self.bert = BERT_model
        
        # simple neural network that convert embedding of the first token to a class
        self.classifier = nn.Sequential(
            nn.Linear(BERT_hidden_size, hidden_layer_size),
            nn.ReLU(),
            nn.Linear(hidden_layer_size, num_classes)
        )
        
    def forward(self, input_ids, attention_mask):
        """
        The following two arguments are tensors from a mini-batch of the input_ids
            and attention_mask returned by the BERT tokenizer.
            
            input_ids: a tensor of shape [batch_size, max_length]
            attention_mask: a tensor of shape [batch_size, max_length]
            
            return: the logits of the sentences in the batch tensor of shape [batch_size, 1, 2]
        """
        # see https://huggingface.co/docs/transformers/model_doc/bert#transformers.BertModel
        # and https://huggingface.co/docs/transformers/main_classes/output
        # for the return value of the forward function of BERT

        z = self.bert.forward(input_ids, attention_mask)
        return torch.softmax(self.classifier(z[1]).unsqueeze(1), dim=-1) 
        
        return logits

In [5]:
# make sure you check the availability of GPU when you set the device ID.
device = torch.device(0)

BATCH_SIZE = 8
num_classes = 7
classifier_hidden_size = 128

## if using MAGIC
BERT_PATH = '/data/badassnlp/bert-base-uncased/'
## if using Google Colab, you need to load the bert model after it downloads the model files.
## 

BERT_hidden_size = 768

N_EPOCHS = 5
CLIP = 1

In [6]:
## 'uncased' means all words are lowered-cased before tokenization
## 'base' means the smaller version of BERT (12 layers, 16 heads)
## un-comment one of the following two options.

# if using MAGIC, load from local BERT folder
tokenizer = BertTokenizer.from_pretrained(BERT_PATH, local_files_only=True)
BERT_model = BertModel.from_pretrained(BERT_PATH, num_labels = 2, output_attentions = False, output_hidden_states = False
).to(device)

## if using Colab, load from automatically downloaded files. Downloading can take half a minute
# tokenizer = BertTokenizer.from_pretrained("bert-base-cased")
# BERT_model = BertModel.from_pretrained("bert-base-cased", num_labels = 2, output_attentions = False, output_hidden_states = False).to(device)


## if using MAGIC
print("creating training corpora ... ")
training_corpora = Corpora(train_list)
print("creating training dataset ... ")
training_dataset = TextClassificationDataSet(training_corpora, tokenizer)
print("creating training iterator ... ")
training_iterator = DataLoader(training_dataset, sampler = RandomSampler(training_dataset), batch_size=BATCH_SIZE)

## if using MAGIC
print("creating test corpora ... ")
dev_corpora = Corpora(test_list)
print("creating test dataset ... ")
dev_dataset = TextClassificationDataSet(dev_corpora, tokenizer)
print("creating test iterator ... ")
dev_iterator = DataLoader(dev_dataset, sampler = SequentialSampler(dev_dataset), batch_size=BATCH_SIZE)

classifier = BERTClassifier(BERT_model, classifier_hidden_size, num_classes).to(device)

Some weights of the model checkpoint at /data/badassnlp/bert-base-uncased/ were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


creating training corpora ... 


100%|██████████| 27180/27180 [00:00<00:00, 1282237.62it/s]

creating training dataset ... 


creating training iterator ... 
creating test corpora ... 


100%|██████████| 6795/6795 [00:00<00:00, 1261856.71it/s]

creating test dataset ... 


creating test iterator ... 


In [7]:
classifier = BERTClassifier(BERT_model, classifier_hidden_size, num_classes).to(device)

In [8]:
classifier.load_state_dict(torch.load("best_model_7classes_balanced_cpu.pt"))

<All keys matched successfully>

In [9]:
from torch import optim
criterion = nn.CrossEntropyLoss(reduction = 'sum')

from tqdm import tqdm

In [10]:
errors = []
confusion_matrix = []

def evaluate(model, iterator, criterion):

    model.eval()
    confusion_matrix.append(torch.zeros(num_classes,num_classes))
    
    for i, batch in tqdm(enumerate(iterator)):
        
        ids = batch[0].to(device)
        msk = batch[1].to(device)
        y = (batch[2]).to(device)
        logits = model.forward(ids,msk)
        loss = criterion(logits.squeeze(1), y) 
        

        for i in range(len(y)):
            row = y[i]
            col = torch.argmax((logits.squeeze(1))[i])
            confusion_matrix[0][row][col] += 1
            if abs(row-col) >= 1:
                errors.append((ids[i], row, col))

In [11]:
evaluate(classifier, dev_iterator, criterion) 

850it [00:24, 34.01it/s]


In [12]:
confusion_matrix

[tensor([[394.,  82., 192.,  47.,  68., 130.,  22.],
         [305., 356., 338., 143., 290., 338., 109.],
         [245., 128., 622.,  51.,  87., 157.,  43.],
         [ 75.,  91.,  68., 407., 103.,  87.,  34.],
         [184., 172., 143., 137., 637., 149.,  67.],
         [ 44.,  15.,  22.,   8.,   6.,  59.,   3.],
         [ 17.,  18.,  17.,  10.,  13.,  16.,  46.]])]

In [13]:
kept

['problems with friends',
 'ongoing depression',
 'breakup with partner',
 'academic pressure',
 'job crisis',
 'Family Issues',
 'Sleep Problems']

In [21]:
import numpy as np
sentence = "It is alright. Thank you"
es = torch.tensor([tokenizer.encode(sentence)])
msk = torch.ones(len(es[0])).unsqueeze(0)
es = es.to(device)
msk = msk.to(device)
logits = classifier.forward(es,msk)
print(logits)
kept[torch.argmax(logits)]

tensor([[[0.0124, 0.5046, 0.0940, 0.1059, 0.0686, 0.2070, 0.0076]]],
       device='cuda:0', grad_fn=<SoftmaxBackward0>)


'ongoing depression'